In [5]:
!pip install tensorflow tensorflow-hub tensorflow-datasets


In [6]:
from google.colab import files

# This will prompt you to upload an image
uploaded = files.upload()


Saving apple.jpeg to apple (1).jpeg


In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the pre-trained object detection model from TensorFlow Hub
model_url = "https://tfhub.dev/tensorflow/efficientdet/lite0/detection/1"
model = hub.load(model_url)

# Load the labels for the COCO dataset
labels = ["???"] * 91
labels_path = tf.keras.utils.get_file(
    'mscoco_label_map.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/mscoco_label_map.txt')
with open(labels_path) as f:
    for line in f.readlines():
        idx, label = line.split(maxsplit=1)
        labels[int(idx)] = label.strip()

# Load an image from the Colab environment
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [512, 512])  # Resize to the input size of the model
    img = img / 255.0  # Normalize image
    return img

# Function to perform object detection
def detect_objects(image):
    image = image[tf.newaxis, ...]  # Add batch dimension
    result = model(image)
    return result

# Display detection results on the image
def display_detections(image_path, result):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = cv2.resize(image, (512, 512))

    boxes = result["detection_boxes"].numpy()[0]
    class_indices = result["detection_classes"].numpy()[0].astype(int)
    scores = result["detection_scores"].numpy()[0]

    for i in range(len(scores)):
        if scores[i] > 0.5:  # Only display detections with confidence above 50%
            ymin, xmin, ymax, xmax = boxes[i]
            xmin = int(xmin * image_resized.shape[1])
            xmax = int(xmax * image_resized.shape[1])
            ymin = int(ymin * image_resized.shape[0])
            ymax = int(ymax * image_resized.shape[0])
            class_name = labels[class_indices[i]]

            # Draw bounding box
            cv2.rectangle(image_resized, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            # Draw label
            cv2.putText(image_resized, class_name, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(image_resized)
    plt.axis('off')
    plt.show()

# Use the uploaded image
for image_path in uploaded.keys():
    img = load_image(image_path)
    result = detect_objects(img)
    display_detections(image_path, result)


Exception: URL fetch failure on https://storage.googleapis.com/download.tensorflow.org/data/mscoco_label_map.txt: 403 -- Forbidden